In [1]:
import sys
sys.path.insert(0,'../')
from MileStone_1_Training_Validation_data_Setup.load_data import load_data_from_ImageFolder

SOURCE_DIR = "../Dataset_Willys_2020/ORGINAL/"
dataset = load_data_from_ImageFolder(root=SOURCE_DIR)


In [2]:
import numpy as np




valid_procentage = 0.15
test_procentage = 0.1
batch_size = 400

n_jobs = 32
n_epochs =10


num_train = len(dataset)
val_indices = list(range(num_train))
valid_split = int(valid_procentage*num_train)



valid_idx = np.random.choice(val_indices, size=valid_split, replace=False)
train_idx = list(set(val_indices)-set(valid_idx))

test_split = int(test_procentage*len(train_idx))
test_indices = list(range(len(train_idx)))
test_idx = np.random.choice(test_indices, size=test_split,replace=False)

train_idx = list(set(test_indices) - set(test_idx))

print(f"total number of images {num_train}, which {len(train_idx)} of them is used for training")
print(f"And {len(valid_idx)} for validation and {len(test_idx)} for testing")

assert len(train_idx)+ len(valid_idx) +len(test_idx) == num_train 

total number of images 9730, which 7444 of them is used for training
And 1459 for validation and 827 for testing


In [3]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


train_sampler = SubsetRandomSampler(train_idx)
train_loader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size,
                          num_workers=n_jobs, pin_memory=True, drop_last=True)

valid_sampler = SubsetRandomSampler(valid_idx)
valid_loader = DataLoader(dataset, sampler=valid_sampler, batch_size=batch_size,
                          num_workers=n_jobs, pin_memory=True, drop_last=True)

test_sampler = SubsetRandomSampler(test_idx)
test_loader = DataLoader(dataset, sampler=test_sampler, batch_size=batch_size,
                          num_workers=n_jobs, pin_memory=True, drop_last=True
                          )

In [4]:
import torch.optim as optim
def train(model, device, train_loaderIn, args):
    optimizer = optim.SGD(model.parameters(), lr=args["lr"], momentum = args["momentum"])
    # optimizer = optim.AdamW(model.parameters(), weight_decay=1e-6)
    for epoch in range(1, args["epochs"]+1):
        train_epoch(epoch=epoch, args=args, model=model,
         device=device, data_loader=train_loaderIn, optimizer=optimizer
         )


In [5]:
from torch.nn import functional as F
def train_epoch(epoch:int, args:dict, model, device, data_loader, optimizer):
    model.train(True)
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        # output_pred = F.log_softmax(output,dim=1)
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % args["log_interval"] == 0:
            print("{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()
                ) 
            )
            

In [6]:
def test(model, device, test_loaderIn):
    test_epoch(model=model, device=device,data_loader=test_loaderIn)
    

In [7]:
def test_epoch(model, device, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data.to(device))
            # output_pred = F.log_softmax(output,dim=1)
            test_loss += F.nll_loss(output, target.to(device),reduction='sum').item()
            pred = output.max(1)[1]
            correct += pred.eq(target.to(device)).sum().item()

    test_loss /=  len(data_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset))
        )


In [4]:
#import the model
from MileStone_3_Create_Neural_Network.WillyNet import WillyNet
import torch
assert torch.cuda.is_available()
torch.backends.cudnn.benchmark=True

willy = WillyNet()
from torchinfo import summary
summary(willy)




Layer (type:depth-idx)                   Param #
WillyNet                                 --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       87,300
│    └─BatchNorm2d: 2-2                  360
│    └─Mish: 2-3                         --
│    └─Dropout2d: 2-4                    --
├─Sequential: 1-2                        --
│    └─Conv2d: 2-5                       1,458,100
│    └─BatchNorm2d: 2-6                  200
│    └─Mish: 2-7                         --
│    └─Dropout2d: 2-8                    --
├─Sequential: 1-3                        --
│    └─Conv2d: 2-9                       1,280,100
│    └─BatchNorm2d: 2-10                 200
│    └─Mish: 2-11                        --
│    └─Dropout2d: 2-12                   --
├─Sequential: 1-4                        --
│    └─Conv2d: 2-13                      864,060
│    └─BatchNorm2d: 2-14                 120
│    └─Mish: 2-15                        --
│    └─Dropout2d: 2-16                   --


In [7]:
args = {"lr": 0.01, "momentum":0.5, "epochs": 5, "log_interval": 3 }
device = torch.device('cuda:0')
willy.to(device)
willy.share_memory()
train(model=willy, device=device, train_loader=train_loader,args=args)


464944	Train Epoch: 1 [0/9730 (0%)]	Loss: -0.214329
464944	Train Epoch: 1 [256/9730 (3%)]	Loss: -0.207512
464944	Train Epoch: 1 [512/9730 (7%)]	Loss: -0.222591
464944	Train Epoch: 1 [768/9730 (10%)]	Loss: -0.220490
464944	Train Epoch: 1 [1024/9730 (14%)]	Loss: -0.347552
464944	Train Epoch: 1 [1280/9730 (17%)]	Loss: -0.329333
464944	Train Epoch: 1 [1536/9730 (21%)]	Loss: -0.348770
464944	Train Epoch: 1 [1792/9730 (24%)]	Loss: -0.347140
464944	Train Epoch: 1 [2048/9730 (28%)]	Loss: -0.324026
464944	Train Epoch: 1 [2304/9730 (31%)]	Loss: -0.356711
464944	Train Epoch: 1 [2560/9730 (34%)]	Loss: -0.376132
464944	Train Epoch: 1 [2816/9730 (38%)]	Loss: -0.368968
464944	Train Epoch: 1 [3072/9730 (41%)]	Loss: -0.393984
464944	Train Epoch: 1 [3328/9730 (45%)]	Loss: -0.432054
464944	Train Epoch: 1 [3584/9730 (48%)]	Loss: -0.411580
464944	Train Epoch: 1 [3840/9730 (52%)]	Loss: -0.448485
464944	Train Epoch: 1 [4096/9730 (55%)]	Loss: -0.500170
464944	Train Epoch: 1 [4352/9730 (59%)]	Loss: -0.487960
4

In [15]:
test(model=willy, device=device, test_loader=train_loader)



Test set: Average loss: -0.7862, Accuracy: 1450/9730 (15%)



In [9]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 5, "log_interval": 3}
device = torch.device('cuda:0')
willy.to(device)
willy.share_memory()
train(model=willy, device=device, train_loader=train_loader, args=args)


487114	Train Epoch: 1 [0/9730 (0%)]	Loss: 3.085736
487114	Train Epoch: 1 [1200/9730 (17%)]	Loss: 3.062003
487114	Train Epoch: 1 [2400/9730 (33%)]	Loss: 2.928991
487114	Train Epoch: 1 [3600/9730 (50%)]	Loss: 2.824156
487114	Train Epoch: 1 [4800/9730 (67%)]	Loss: 2.825988
487114	Train Epoch: 1 [6000/9730 (83%)]	Loss: 2.807476
487114	Train Epoch: 2 [0/9730 (0%)]	Loss: 2.797379
487114	Train Epoch: 2 [1200/9730 (17%)]	Loss: 2.833917
487114	Train Epoch: 2 [2400/9730 (33%)]	Loss: 2.821617
487114	Train Epoch: 2 [3600/9730 (50%)]	Loss: 2.795099
487114	Train Epoch: 2 [4800/9730 (67%)]	Loss: 2.778999
487114	Train Epoch: 2 [6000/9730 (83%)]	Loss: 2.774713
487114	Train Epoch: 3 [0/9730 (0%)]	Loss: 2.767551
487114	Train Epoch: 3 [1200/9730 (17%)]	Loss: 2.746999
487114	Train Epoch: 3 [2400/9730 (33%)]	Loss: 2.752237
487114	Train Epoch: 3 [3600/9730 (50%)]	Loss: 2.728916
487114	Train Epoch: 3 [4800/9730 (67%)]	Loss: 2.768703


In [ ]:
test(model=willy, device=device, test_loader=train_loader)
test(model=willy, device=device,test_loader=valid_loader)

In [9]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 5, "log_interval": 3}
device = torch.device('cuda:0')
willy.to(device)
willy.share_memory()
train(model=willy, device=device, train_loader=train_loader, args=args)


496275	Train Epoch: 1 [0/9730 (0%)]	Loss: 3.167415
496275	Train Epoch: 1 [1200/9730 (17%)]	Loss: 3.015418
496275	Train Epoch: 1 [2400/9730 (33%)]	Loss: 2.904104
496275	Train Epoch: 1 [3600/9730 (50%)]	Loss: 2.813510
496275	Train Epoch: 1 [4800/9730 (67%)]	Loss: 2.781481
496275	Train Epoch: 1 [6000/9730 (83%)]	Loss: 2.756980
496275	Train Epoch: 2 [0/9730 (0%)]	Loss: 2.762283
496275	Train Epoch: 2 [1200/9730 (17%)]	Loss: 2.762573
496275	Train Epoch: 2 [2400/9730 (33%)]	Loss: 2.739838
496275	Train Epoch: 2 [3600/9730 (50%)]	Loss: 2.738945
496275	Train Epoch: 2 [4800/9730 (67%)]	Loss: 2.733472
496275	Train Epoch: 2 [6000/9730 (83%)]	Loss: 2.730531
496275	Train Epoch: 3 [0/9730 (0%)]	Loss: 2.741371
496275	Train Epoch: 3 [1200/9730 (17%)]	Loss: 2.725900
496275	Train Epoch: 3 [2400/9730 (33%)]	Loss: 2.732535
496275	Train Epoch: 3 [3600/9730 (50%)]	Loss: 2.731039
496275	Train Epoch: 3 [4800/9730 (67%)]	Loss: 2.715538
496275	Train Epoch: 3 [6000/9730 (83%)]	Loss: 2.694800
496275	Train Epoch: 4 

In [10]:
test(model=willy, device=device, test_loader=train_loader)
test(model=willy, device=device, test_loader=valid_loader)



Test set: Average loss: 1.9487, Accuracy: 1347/9730 (14%)


Test set: Average loss: 0.3256, Accuracy: 238/9730 (2%)



In [10]:
test(model=willy, device=device, test_loader=train_loader)
test(model=willy, device=device, test_loader=valid_loader)



Test set: Average loss: 2.1293, Accuracy: 22/9730 (0%)


Test set: Average loss: 0.3552, Accuracy: 1/9730 (0%)



In [12]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 5, "log_interval": 3}
device = torch.device('cuda:0')
willy.to(device)
willy.share_memory()
train(model=willy, device=device, train_loaderIn=train_loader, args=args)


515045	Train Epoch: 1 [0/9730 (0%)]	Loss: 3.370296
515045	Train Epoch: 1 [1200/9730 (17%)]	Loss: 3.204740
515045	Train Epoch: 1 [2400/9730 (33%)]	Loss: 3.100020
515045	Train Epoch: 1 [3600/9730 (50%)]	Loss: 3.009669
515045	Train Epoch: 1 [4800/9730 (67%)]	Loss: 2.976737
515045	Train Epoch: 1 [6000/9730 (83%)]	Loss: 2.938450
515045	Train Epoch: 2 [0/9730 (0%)]	Loss: 2.910660
515045	Train Epoch: 2 [1200/9730 (17%)]	Loss: 2.922548
515045	Train Epoch: 2 [2400/9730 (33%)]	Loss: 2.840514
515045	Train Epoch: 2 [3600/9730 (50%)]	Loss: 2.803405
515045	Train Epoch: 2 [4800/9730 (67%)]	Loss: 2.894019
515045	Train Epoch: 2 [6000/9730 (83%)]	Loss: 2.852977
515045	Train Epoch: 3 [0/9730 (0%)]	Loss: 2.825532
515045	Train Epoch: 3 [1200/9730 (17%)]	Loss: 2.793926
515045	Train Epoch: 3 [2400/9730 (33%)]	Loss: 2.890319
515045	Train Epoch: 3 [3600/9730 (50%)]	Loss: 2.801518
515045	Train Epoch: 3 [4800/9730 (67%)]	Loss: 2.830374
515045	Train Epoch: 3 [6000/9730 (83%)]	Loss: 2.795450
515045	Train Epoch: 4 

In [13]:

test(model=willy, device=device, test_loader=train_loader)
test(model=willy, device=device, test_loader=valid_loader)



Test set: Average loss: 1.9835, Accuracy: 1301/9730 (13%)


Test set: Average loss: 0.3285, Accuracy: 203/9730 (2%)



In [8]:
import torch.nn as nn
from torchinfo import summary
from torch.nn import functional as F
import os
import torch

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5, dilation=2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, dilation=2)
        self.conv3 = nn.Conv2d(20, 30, kernel_size=5, dilation=2)
        self.conv4 = nn.Conv2d(30, 40, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.conv3_drop = nn.Dropout2d()
        self.conv4_drop = nn.Dropout2d()
        # self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1440, 720)
        self.fc2 = nn.Linear(720, 360)
        self.fc3 = nn.Linear(360, 180)
        self.fc4 = nn.Linear(180, 18)

    def forward(self, x):
        x = F.mish(F.max_pool2d(self.conv1(x), 2))
        x = F.mish(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.mish(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.mish(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))

        #x = x.view(-1, 7200)
        x = self.flatten(x)
        x = F.mish(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.mish(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = F.mish(self.fc3(x))
        x = F.dropout(x, training=self.training)

        x = self.fc4(x)
        return F.log_softmax(x, dim=1)
willy = Net()
summary(willy,input_size=(400,3,198,198))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      --                        --
├─Conv2d: 1-1                            [400, 10, 190, 190]       760
├─Conv2d: 1-2                            [400, 20, 87, 87]         5,020
├─Dropout2d: 1-3                         [400, 20, 87, 87]         --
├─Conv2d: 1-4                            [400, 30, 35, 35]         15,030
├─Dropout2d: 1-5                         [400, 30, 35, 35]         --
├─Conv2d: 1-6                            [400, 40, 13, 13]         30,040
├─Dropout2d: 1-7                         [400, 40, 13, 13]         --
├─Flatten: 1-8                           [400, 1440]               --
├─Linear: 1-9                            [400, 720]                1,037,520
├─Linear: 1-10                           [400, 360]                259,560
├─Linear: 1-11                           [400, 180]                64,980
├─Linear: 1-12                           [400, 18]       

In [13]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 50, "log_interval": 3}
#device = torch.device('cuda:0')
#willy.to(device)
#willy.share_memory()


In [10]:
test(model=willy, device=device, test_loaderIn=train_loader)
test(model=willy, device=device, test_loaderIn=valid_loader)



Test set: Average loss: 2.1395, Accuracy: 0/9730 (0%)


Test set: Average loss: 0.3574, Accuracy: 9/9730 (0%)



In [11]:
# args = {"lr": 0.01, "momentum": 0.5, "epochs": 5, "log_interval": 3}
# device = torch.device('cuda:0')
# willy.to(device)
# willy.share_memory()
train(model=willy, device=device, train_loaderIn=train_loader, args=args)


557596	Train Epoch: 1 [0/9730 (0%)]	Loss: 2.889803
557596	Train Epoch: 1 [1200/9730 (17%)]	Loss: 2.893542
557596	Train Epoch: 1 [2400/9730 (33%)]	Loss: 2.885046
557596	Train Epoch: 1 [3600/9730 (50%)]	Loss: 2.880883
557596	Train Epoch: 1 [4800/9730 (67%)]	Loss: 2.881128
557596	Train Epoch: 1 [6000/9730 (83%)]	Loss: 2.882294
557596	Train Epoch: 2 [0/9730 (0%)]	Loss: 2.876898
557596	Train Epoch: 2 [1200/9730 (17%)]	Loss: 2.874690
557596	Train Epoch: 2 [2400/9730 (33%)]	Loss: 2.870163
557596	Train Epoch: 2 [3600/9730 (50%)]	Loss: 2.869845
557596	Train Epoch: 2 [4800/9730 (67%)]	Loss: 2.869622
557596	Train Epoch: 2 [6000/9730 (83%)]	Loss: 2.863748
557596	Train Epoch: 3 [0/9730 (0%)]	Loss: 2.864494
557596	Train Epoch: 3 [1200/9730 (17%)]	Loss: 2.861001
557596	Train Epoch: 3 [2400/9730 (33%)]	Loss: 2.859577
557596	Train Epoch: 3 [3600/9730 (50%)]	Loss: 2.847371
557596	Train Epoch: 3 [4800/9730 (67%)]	Loss: 2.852261
557596	Train Epoch: 3 [6000/9730 (83%)]	Loss: 2.851653
557596	Train Epoch: 4 

In [14]:
train(model=willy, device=device, train_loaderIn=train_loader, args=args)


557596	Train Epoch: 1 [0/9730 (0%)]	Loss: 2.817371
557596	Train Epoch: 1 [1200/9730 (17%)]	Loss: 2.809611
557596	Train Epoch: 1 [2400/9730 (33%)]	Loss: 2.802074
557596	Train Epoch: 1 [3600/9730 (50%)]	Loss: 2.799877
557596	Train Epoch: 1 [4800/9730 (67%)]	Loss: 2.799095
557596	Train Epoch: 1 [6000/9730 (83%)]	Loss: 2.784458
557596	Train Epoch: 2 [0/9730 (0%)]	Loss: 2.787398
557596	Train Epoch: 2 [1200/9730 (17%)]	Loss: 2.773044
557596	Train Epoch: 2 [2400/9730 (33%)]	Loss: 2.773338
557596	Train Epoch: 2 [3600/9730 (50%)]	Loss: 2.772168
557596	Train Epoch: 2 [4800/9730 (67%)]	Loss: 2.758266
557596	Train Epoch: 2 [6000/9730 (83%)]	Loss: 2.771692
557596	Train Epoch: 3 [0/9730 (0%)]	Loss: 2.722183
557596	Train Epoch: 3 [1200/9730 (17%)]	Loss: 2.722759
557596	Train Epoch: 3 [2400/9730 (33%)]	Loss: 2.747673
557596	Train Epoch: 3 [3600/9730 (50%)]	Loss: 2.729829
557596	Train Epoch: 3 [4800/9730 (67%)]	Loss: 2.722520
557596	Train Epoch: 3 [6000/9730 (83%)]	Loss: 2.728061
557596	Train Epoch: 4 

In [12]:
test(model=willy, device=device, test_loaderIn=train_loader)
test(model=willy, device=device, test_loaderIn=valid_loader)



Test set: Average loss: 2.0831, Accuracy: 928/9730 (10%)


Test set: Average loss: 0.3488, Accuracy: 137/9730 (1%)



In [9]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 50, "log_interval": 3}
device = torch.device('cuda:0')
willy.to(device)
willy.share_memory()
test(model=willy, device=device, test_loaderIn=train_loader)
test(model=willy, device=device, test_loaderIn=valid_loader)



Test set: Average loss: 2.1439, Accuracy: 892/9730 (9%)


Test set: Average loss: 0.3570, Accuracy: 141/9730 (1%)



In [10]:
train(model=willy, device=device, train_loaderIn=train_loader, args=args)


43922	Train Epoch: 1 [0/9730 (0%)]	Loss: 3.439215
43922	Train Epoch: 1 [1200/9730 (17%)]	Loss: 3.115792
43922	Train Epoch: 1 [2400/9730 (33%)]	Loss: 2.923263
43922	Train Epoch: 1 [3600/9730 (50%)]	Loss: 2.908574
43922	Train Epoch: 1 [4800/9730 (67%)]	Loss: 2.867324
43922	Train Epoch: 1 [6000/9730 (83%)]	Loss: 2.890974
43922	Train Epoch: 2 [0/9730 (0%)]	Loss: 2.786769
43922	Train Epoch: 2 [1200/9730 (17%)]	Loss: 2.764891
43922	Train Epoch: 2 [2400/9730 (33%)]	Loss: 2.757283
43922	Train Epoch: 2 [3600/9730 (50%)]	Loss: 2.771966
43922	Train Epoch: 2 [4800/9730 (67%)]	Loss: 2.726995
43922	Train Epoch: 2 [6000/9730 (83%)]	Loss: 2.715253
43922	Train Epoch: 3 [0/9730 (0%)]	Loss: 2.712765
43922	Train Epoch: 3 [1200/9730 (17%)]	Loss: 2.708520
43922	Train Epoch: 3 [2400/9730 (33%)]	Loss: 2.651708
43922	Train Epoch: 3 [3600/9730 (50%)]	Loss: 2.665691
43922	Train Epoch: 3 [4800/9730 (67%)]	Loss: 2.740607
43922	Train Epoch: 3 [6000/9730 (83%)]	Loss: 2.683159
43922	Train Epoch: 4 [0/9730 (0%)]	Loss:

In [ ]:
test(model=willy, device=device, test_loaderIn=train_loader)
test(model=willy, device=device, test_loaderIn=valid_loader)

In [8]:
from MileStone_3_Create_Neural_Network.WillyAlexNet import MyAlexNet
import torch
from torchinfo import summary
willyAlexNet = MyAlexNet()

# for param in willyAlexNet.parameters():
#     param.requires_grad = False

# for param in willyAlexNet.WillyAlexNet.classifier[-1].parameters():
    # param.requires_grad = True
summary(willyAlexNet, input_size=(batch_size,3,198,198))
#print(willyAlexNet)

Layer (type:depth-idx)                   Output Shape              Param #
MyAlexNet                                --                        --
├─AlexNet: 1-1                           [400, 18]                 --
│    └─Sequential: 2-1                   [400, 256, 5, 5]          --
│    │    └─Conv2d: 3-1                  [400, 64, 48, 48]         23,296
│    │    └─ReLU: 3-2                    [400, 64, 48, 48]         --
│    │    └─MaxPool2d: 3-3               [400, 64, 23, 23]         --
│    │    └─Conv2d: 3-4                  [400, 192, 23, 23]        307,392
│    │    └─ReLU: 3-5                    [400, 192, 23, 23]        --
│    │    └─MaxPool2d: 3-6               [400, 192, 11, 11]        --
│    │    └─Conv2d: 3-7                  [400, 384, 11, 11]        663,936
│    │    └─ReLU: 3-8                    [400, 384, 11, 11]        --
│    │    └─Conv2d: 3-9                  [400, 256, 11, 11]        884,992
│    │    └─ReLU: 3-10                   [400, 256, 11, 11]       

In [9]:
summary(willyAlexNet)

Layer (type:depth-idx)                   Param #
MyAlexNet                                --
├─AlexNet: 1-1                           --
│    └─Sequential: 2-1                   --
│    │    └─Conv2d: 3-1                  23,296
│    │    └─ReLU: 3-2                    --
│    │    └─MaxPool2d: 3-3               --
│    │    └─Conv2d: 3-4                  307,392
│    │    └─ReLU: 3-5                    --
│    │    └─MaxPool2d: 3-6               --
│    │    └─Conv2d: 3-7                  663,936
│    │    └─ReLU: 3-8                    --
│    │    └─Conv2d: 3-9                  884,992
│    │    └─ReLU: 3-10                   --
│    │    └─Conv2d: 3-11                 590,080
│    │    └─ReLU: 3-12                   --
│    │    └─MaxPool2d: 3-13              --
│    └─AdaptiveAvgPool2d: 2-2            --
│    └─Sequential: 2-3                   --
│    │    └─Dropout: 3-14                --
│    │    └─Linear: 3-15                 37,752,832
│    │    └─ReLU: 3-16                 

In [10]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 50, "log_interval": 3}
device = torch.device('cuda:0')
willyAlexNet.to(device)
willyAlexNet.share_memory()

test(model=willyAlexNet, device=device, test_loaderIn=train_loader)
test(model=willyAlexNet, device=device, test_loaderIn=valid_loader)



Test set: Average loss: 2.4038, Accuracy: 273/9730 (3%)


Test set: Average loss: 0.3943, Accuracy: 56/9730 (1%)



In [11]:
train(model=willyAlexNet, device=device, train_loaderIn=train_loader, args=args)


33582	Train Epoch: 1 [0/9730 (0%)]	Loss: 3.438911
33582	Train Epoch: 1 [1200/9730 (17%)]	Loss: 2.364406
33582	Train Epoch: 1 [2400/9730 (33%)]	Loss: 2.194092
33582	Train Epoch: 1 [3600/9730 (50%)]	Loss: 2.232552
33582	Train Epoch: 1 [4800/9730 (67%)]	Loss: 2.136112
33582	Train Epoch: 1 [6000/9730 (83%)]	Loss: 2.155311
33582	Train Epoch: 2 [0/9730 (0%)]	Loss: 2.019054
33582	Train Epoch: 2 [1200/9730 (17%)]	Loss: 2.058074
33582	Train Epoch: 2 [2400/9730 (33%)]	Loss: 2.036019
33582	Train Epoch: 2 [3600/9730 (50%)]	Loss: 2.015141
33582	Train Epoch: 2 [4800/9730 (67%)]	Loss: 1.982431
33582	Train Epoch: 2 [6000/9730 (83%)]	Loss: 1.920383
33582	Train Epoch: 3 [0/9730 (0%)]	Loss: 1.892114
33582	Train Epoch: 3 [1200/9730 (17%)]	Loss: 1.951307
33582	Train Epoch: 3 [2400/9730 (33%)]	Loss: 1.865821
33582	Train Epoch: 3 [3600/9730 (50%)]	Loss: 1.917925
33582	Train Epoch: 3 [4800/9730 (67%)]	Loss: 1.867115
33582	Train Epoch: 3 [6000/9730 (83%)]	Loss: 1.906361
33582	Train Epoch: 4 [0/9730 (0%)]	Loss:

In [12]:
test(model=willyAlexNet, device=device, test_loaderIn=train_loader)
test(model=willyAlexNet, device=device, test_loaderIn=valid_loader)



Test set: Average loss: 0.5723, Accuracy: 5311/9730 (55%)


Test set: Average loss: 0.1559, Accuracy: 775/9730 (8%)



In [13]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 50, "log_interval": 3}
train(model=willyAlexNet, device=device, train_loaderIn=train_loader, args=args)


33582	Train Epoch: 1 [0/9730 (0%)]	Loss: 0.968747
33582	Train Epoch: 1 [1200/9730 (17%)]	Loss: 0.903528
33582	Train Epoch: 1 [2400/9730 (33%)]	Loss: 0.919176
33582	Train Epoch: 1 [3600/9730 (50%)]	Loss: 0.851960
33582	Train Epoch: 1 [4800/9730 (67%)]	Loss: 0.987713
33582	Train Epoch: 1 [6000/9730 (83%)]	Loss: 0.990164
33582	Train Epoch: 2 [0/9730 (0%)]	Loss: 0.914763
33582	Train Epoch: 2 [1200/9730 (17%)]	Loss: 0.906578
33582	Train Epoch: 2 [2400/9730 (33%)]	Loss: 0.894332
33582	Train Epoch: 2 [3600/9730 (50%)]	Loss: 0.955034
33582	Train Epoch: 2 [4800/9730 (67%)]	Loss: 1.044238
33582	Train Epoch: 2 [6000/9730 (83%)]	Loss: 0.839121
33582	Train Epoch: 3 [0/9730 (0%)]	Loss: 0.861534
33582	Train Epoch: 3 [1200/9730 (17%)]	Loss: 0.849830
33582	Train Epoch: 3 [2400/9730 (33%)]	Loss: 0.967214
33582	Train Epoch: 3 [3600/9730 (50%)]	Loss: 0.921844
33582	Train Epoch: 3 [4800/9730 (67%)]	Loss: 0.846225
33582	Train Epoch: 3 [6000/9730 (83%)]	Loss: 0.932787
33582	Train Epoch: 4 [0/9730 (0%)]	Loss:

In [14]:
test(model=willyAlexNet, device=device, test_loaderIn=train_loader)
test(model=willyAlexNet, device=device, test_loaderIn=valid_loader)



Test set: Average loss: 0.4114, Accuracy: 5764/9730 (59%)


Test set: Average loss: 0.1484, Accuracy: 834/9730 (9%)



In [8]:
from MileStone_3_Create_Neural_Network.WillysShuffleNet_v2_x1_0 import myShuffleNet_v2_x1_0
import torch
from torchinfo import summary
willyShuffleNet = myShuffleNet_v2_x1_0()

# for param in willyAlexNet.parameters():
#     param.requires_grad = False

# for param in willyAlexNet.WillyAlexNet.classifier[-1].parameters():
    # param.requires_grad = True
summary(willyShuffleNet, input_size=(batch_size,3,198,198))
#print(willyAlexNet)

Layer (type:depth-idx)                        Output Shape              Param #
├─ShuffleNetV2: 1-1                           [400, 18]                 --
│    └─Sequential: 2-1                        [400, 24, 99, 99]         --
│    │    └─Conv2d: 3-1                       [400, 24, 99, 99]         648
│    │    └─BatchNorm2d: 3-2                  [400, 24, 99, 99]         48
│    │    └─ReLU: 3-3                         [400, 24, 99, 99]         --
│    └─MaxPool2d: 2-2                         [400, 24, 50, 50]         --
│    └─Sequential: 2-3                        [400, 48, 25, 25]         --
│    │    └─InvertedResidual: 3-4             [400, 48, 25, 25]         2,400
│    │    └─InvertedResidual: 3-5             [400, 48, 25, 25]         1,512
│    │    └─InvertedResidual: 3-6             [400, 48, 25, 25]         1,512
│    │    └─InvertedResidual: 3-7             [400, 48, 25, 25]         1,512
│    └─Sequential: 2-4                        [400, 96, 13, 13]         --
│    │ 

In [9]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 50, "log_interval": 3}
device = torch.device('cuda:0')
willyShuffleNet.to(device)
willyShuffleNet.share_memory()

test(model=willyShuffleNet, device=device, test_loaderIn=train_loader)
test(model=willyShuffleNet, device=device, test_loaderIn=valid_loader)



Test set: Average loss: 2.1400, Accuracy: 432/9730 (4%)


Test set: Average loss: 0.3572, Accuracy: 69/9730 (1%)



In [11]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 50, "log_interval": 3}
train(model=willyShuffleNet, device=device, train_loaderIn=train_loader, args=args)

76559	Train Epoch: 1 [0/9730 (0%)]	Loss: 2.892886
76559	Train Epoch: 1 [1200/9730 (17%)]	Loss: 2.891061
76559	Train Epoch: 1 [2400/9730 (33%)]	Loss: 2.887151
76559	Train Epoch: 1 [3600/9730 (50%)]	Loss: 2.883877
76559	Train Epoch: 1 [4800/9730 (67%)]	Loss: 2.881113
76559	Train Epoch: 1 [6000/9730 (83%)]	Loss: 2.878371
76559	Train Epoch: 2 [0/9730 (0%)]	Loss: 2.875867
76559	Train Epoch: 2 [1200/9730 (17%)]	Loss: 2.872226
76559	Train Epoch: 2 [2400/9730 (33%)]	Loss: 2.869443
76559	Train Epoch: 2 [3600/9730 (50%)]	Loss: 2.868200
76559	Train Epoch: 2 [4800/9730 (67%)]	Loss: 2.865467
76559	Train Epoch: 2 [6000/9730 (83%)]	Loss: 2.860615
76559	Train Epoch: 3 [0/9730 (0%)]	Loss: 2.858515
76559	Train Epoch: 3 [1200/9730 (17%)]	Loss: 2.854532
76559	Train Epoch: 3 [2400/9730 (33%)]	Loss: 2.853480
76559	Train Epoch: 3 [3600/9730 (50%)]	Loss: 2.850220
76559	Train Epoch: 3 [4800/9730 (67%)]	Loss: 2.848416
76559	Train Epoch: 3 [6000/9730 (83%)]	Loss: 2.847564
76559	Train Epoch: 4 [0/9730 (0%)]	Loss:

In [13]:
test(model=willyShuffleNet, device=device, test_loaderIn=train_loader)
test(model=willyShuffleNet, device=device, test_loaderIn=valid_loader)


Test set: Average loss: 1.8702, Accuracy: 1208/9730 (12%)


Test set: Average loss: 0.3170, Accuracy: 165/9730 (2%)



In [14]:
args = {"lr": 0.01, "momentum": 0.5, "epochs": 50, "log_interval": 3}
train(model=willyShuffleNet, device=device,
      train_loaderIn=train_loader, args=args)


76559	Train Epoch: 1 [0/9730 (0%)]	Loss: 2.558404
76559	Train Epoch: 1 [1200/9730 (17%)]	Loss: 2.498841
76559	Train Epoch: 1 [2400/9730 (33%)]	Loss: 2.517309
76559	Train Epoch: 1 [3600/9730 (50%)]	Loss: 2.561944
76559	Train Epoch: 1 [4800/9730 (67%)]	Loss: 2.557342
76559	Train Epoch: 1 [6000/9730 (83%)]	Loss: 2.522970
76559	Train Epoch: 2 [0/9730 (0%)]	Loss: 2.528353
76559	Train Epoch: 2 [1200/9730 (17%)]	Loss: 2.514053
76559	Train Epoch: 2 [2400/9730 (33%)]	Loss: 2.533762
76559	Train Epoch: 2 [3600/9730 (50%)]	Loss: 2.531390
76559	Train Epoch: 2 [4800/9730 (67%)]	Loss: 2.516706
76559	Train Epoch: 2 [6000/9730 (83%)]	Loss: 2.534300
76559	Train Epoch: 3 [0/9730 (0%)]	Loss: 2.535845
76559	Train Epoch: 3 [1200/9730 (17%)]	Loss: 2.503897
76559	Train Epoch: 3 [2400/9730 (33%)]	Loss: 2.541872
76559	Train Epoch: 3 [3600/9730 (50%)]	Loss: 2.528813
76559	Train Epoch: 3 [4800/9730 (67%)]	Loss: 2.529763
76559	Train Epoch: 3 [6000/9730 (83%)]	Loss: 2.546840
76559	Train Epoch: 4 [0/9730 (0%)]	Loss:

In [15]:
test(model=willyShuffleNet, device=device, test_loaderIn=train_loader)
test(model=willyShuffleNet, device=device, test_loaderIn=valid_loader)


Test set: Average loss: 1.7504, Accuracy: 1527/9730 (16%)


Test set: Average loss: 0.3011, Accuracy: 186/9730 (2%)



In [ ]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F


def train(rank, args, model, device, dataset, dataloader_kwargs):
    train_loader = torch.utils.data.DataLoader(dataset, **dataloader_kwargs)

    optimizer = optim.SGD(model.parameters(), lr=args.lr,
                          momentum=args.momentum)
    for epoch in range(1, args.epochs + 1):
        train_epoch(epoch, args, model, device, train_loader, optimizer)


def test(args, model, device, dataset, dataloader_kwargs):
    torch.manual_seed(args.seed)
    test_loader = torch.utils.data.DataLoader(dataset, **dataloader_kwargs)
    test_epoch(model, device, test_loader)


def train_epoch(epoch, args, model, device, data_loader, optimizer):
    model.train()
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()))
            if args.dry_run:
                break


def test_epoch(model, device, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data.to(device))
            # sum up batch loss
            test_loss += F.nll_loss(output, target.to(device),
                                    reduction='sum').item()
            pred = output.max(1)[1]  # get the index of the max log-probability
            correct += pred.eq(target.to(device)).sum().item()

    test_loss /= len(data_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))
